In [23]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

In [24]:
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

import json

import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np

from tqdm import tqdm

import re

from pprint import pprint

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
MODEL_CHECKPOINT = "facebook/bart-base"
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 512

In [27]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [28]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [29]:
train_lists = []
label_lists = []
with open("./bigdata/utterance_persona_dpr.jsonl") as f:
    for json_line in f:
        json_file = json.loads(json_line)
        prev_strs = ""
        dic_list=json_file["extracted_persona_list"]
        for i,dic in enumerate(dic_list):
            if i!=0 and i!=len(dic_list)-1:
                prev_strs += dic["utterance"] + " " 
                train_lists.append(prev_strs + " " + dic["3persona"][0][0]) #[0][1]이 점수
            if i!=0 and i!=1:
                label_lists.append(dic["utterance"])


In [30]:
lengdata=len(train_lists)

In [31]:
valid_lists = train_lists[int(lengdata*0.7):int(lengdata*0.9)]
valid_label_lists = label_lists[int(lengdata*0.7):int(lengdata*0.9)]

test_lists = train_lists[int(lengdata*0.9):]
test_label_lists = label_lists[int(lengdata*0.9):]

train_lists = train_lists[:int(lengdata*0.7)]
train_label_lists = label_lists[:int(lengdata*0.7)]

In [32]:
class bf_Dataset(Dataset):
    def __init__(self, dataset, labelset, tokenizer):
        self.tokenizer = tokenizer
        self.inputs = []
        self.outputs = []
        for data,label in zip(dataset,labelset):
                self.inputs.append(data)
                self.outputs.append(label)
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        embeddings = self.tokenizer(self.inputs[idx], max_length=MAX_INPUT_LENGTH, truncation=True)
        embeddings['labels'] = self.tokenizer(self.outputs[idx], max_length=MAX_TARGET_LENGTH, truncation=True, return_attention_mask=False)["input_ids"]
        return embeddings

In [33]:
train_ds = bf_Dataset(dataset=train_lists, labelset = train_label_lists,tokenizer=tokenizer)
val_ds = bf_Dataset(dataset=valid_lists ,labelset = valid_label_lists ,tokenizer=tokenizer)
test_ds = bf_Dataset(dataset=test_lists ,labelset = test_label_lists ,tokenizer=tokenizer)

In [34]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

In [35]:
model_name = MODEL_CHECKPOINT.split("/")[-1]

In [36]:
batch_size = 16
epochs = 40


args = Seq2SeqTrainingArguments(
    f"./bigdata/{model_name}-finetuned-msc_2",
    evaluation_strategy = "steps",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    save_steps=1600,
    eval_steps=1600,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
)

In [37]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [38]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [39]:
trainer.train()

/home/leadawon5/decs_jupyter_lab/venvs/bartvenv/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1600,2.774600,2.447695
3200,2.643100,2.344550
4800,2.571600,2.264845
6400,2.519100,2.200216
8000,2.444600,2.137277
9600,2.365000,2.081566
11200,2.339600,2.026450
12800,2.301100,1.972325
14400,2.238600,1.923473
16000,2.185200,1.874129


TrainOutput(global_step=86360, training_loss=1.7835319900689384, metrics={'train_runtime': 16478.8267, 'train_samples_per_second': 83.824, 'train_steps_per_second': 5.241, 'total_flos': 3.46040117789184e+17, 'train_loss': 1.7835319900689384, 'epoch': 40.0})

In [18]:
assert False

AssertionError: 

In [19]:
model = AutoModelForSeq2SeqLM.from_pretrained(f"./bigdata/{model_name}-finetuned-msc/checkpoint-20800")

model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [21]:
from rouge import Rouge
rouge = Rouge()

filename = "./output_persona.txt"
with open(filename,"w") as f:
    pass
pp = 0
pred_arr = []
ref_arr = []
with torch.no_grad():
    model.to(device)
    for i,embeddings in enumerate(test_ds):
        
        output = model.generate(torch.tensor([embeddings["input_ids"]]).to(device))[0]    
        pred = tokenizer.decode(output.cpu(), skip_special_tokens=True)  
        gt = tokenizer.decode(torch.tensor(embeddings["labels"]), skip_special_tokens=True)
    
        
        
        stringt = "input_ids"
        with open(filename,"a") as f:
            f.write(f"{i} 번째 문장\ninput_ids : \n{tokenizer.decode(embeddings[stringt],skip_special_tokens=True)}\nprediction: \n{pred}\ngt        : \n{gt}\n\n")
        pp += rouge.get_scores(pred,gt,avg=True)['rouge-1']['f']


In [22]:
pp /= len(test_ds)
print(pp)

0.26489487094005554
